In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import openai
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from openai.error import RateLimitError

# Set your OpenAI API key
openai.api_key = '****'  # Replace with your OpenAI API key

# CSS
st.markdown("""
    <style>
    .main {
        background-color: #f0f2f6;
        padding: 20px;
    }
    .stButton button {
        background-color: #1f77b4;
        color: white;
        font-size: 16px;
        border-radius: 8px;
        padding: 10px;
    }
    .stTextArea textarea {
        background-color: #e3f2fd;
        border-radius: 8px;
    }
    </style>
    """, unsafe_allow_html=True)

# Helper function to determine the type of each column
def data_profile(df):
    profile = {}
    for col in df.columns:
        dtype = df[col].dtype
        if pd.api.types.is_numeric_dtype(df[col]):
            unique_values = df[col].nunique()
            if unique_values > 10:
                profile[col] = "Continuous"
            else:
                profile[col] = "Discrete"
        elif pd.api.types.is_categorical_dtype(df[col]) or df[col].dtype == 'object':
            profile[col] = "Categorical"
        else:
            profile[col] = "Other"
    return profile

# Missing values check
def missing_values(df):
    return df.isnull().sum()

# Unique values check
def unique_values(df):
    return df.nunique()

# Descriptive statistics
def summary_stats(df):
    return df.describe()

# Cardinality check for categorical columns
def cardinality(df):
    return df.select_dtypes(include=['object']).nunique()

# Correlation matrix (Fix: Only select numerical columns)
def correlation_matrix(df):
    numeric_df = df.select_dtypes(include=[np.number])  
    if numeric_df.empty:
        return "No numerical columns available for correlation."
    return numeric_df.corr()

# Outlier detection using IQR (Fix: Only numeric columns)
def detect_outliers(df):
    numeric_df = df.select_dtypes(include=[np.number])  
    if numeric_df.empty:
        return "No numerical columns available for outlier detection."
    
    Q1 = numeric_df.quantile(0.25)
    Q3 = numeric_df.quantile(0.75)
    IQR = Q3 - Q1
    return (numeric_df < (Q1 - 1.5 * IQR)) | (numeric_df > (Q3 + 1.5 * IQR))

# Duplicate rows check
def duplicate_rows(df):
    return df.duplicated().sum()

# Memory usage check
def memory_usage(df):
    return df.memory_usage(deep=True)

# Zero and constant values check
def zero_constant_check(df):
    zero_columns = (df == 0).sum()
    constant_columns = df.nunique() == 1
    return zero_columns, constant_columns

# Time series decomposition (for time-series data)
def time_series_analysis(df, column):
    df[column] = pd.to_datetime(df[column])
    decomposition = seasonal_decompose(df[column], model='additive')
    return decomposition

# Histogram visualization for continuous variables
def plot_histograms(df):
    numeric_df = df.select_dtypes(include=[np.number]) 
    continuous_columns = [col for col in numeric_df.columns if numeric_df[col].nunique() > 10]  
    if not continuous_columns:
        st.write("No continuous variables available for histogram.")
    else:
        for col in continuous_columns:
            st.subheader(f"Histogram for {col}")
            fig, ax = plt.subplots()
            ax.hist(numeric_df[col].dropna(), bins=20, color='blue', edgecolor='black')
            ax.set_title(f"Distribution of {col}")
            ax.set_xlabel(col)
            ax.set_ylabel("Frequency")
            st.pyplot(fig)

# Function to send data and description to OpenAI's API
def get_openai_response(dataset, description):
    # Limit the dataset to a reasonable size
    dataset_sample = dataset.iloc[:50, :10]  # Limit to 50 rows and 10 columns

    # Prepare dataset as a JSON string
    dataset_json = dataset_sample.to_json()

    # Prepare the message for OpenAI GPT model
    messages = [
        {"role": "system", "content": "You are a data analysis assistant."},
        {"role": "user", "content": f"Dataset: {dataset_json}\n\nDescription: {description}"}
    ]

    try:
        # Send the API request
        response = openai.ChatCompletion.create(
            model="****", # Replace with the model you intend to use
            messages=messages
        )
        return response.choices[0].message['content']

    except RateLimitError:
        # Display a friendly message if the data is too large
        return "The dataset is too large to process. Please reduce the size or sample the data."

# Streamlit app interface
st.title("🔍 Comprehensive Data Profiler")
st.sidebar.title("🔧 Settings & Upload")
st.sidebar.markdown("Upload your dataset and choose from various profiling options.")

# File upload option
uploaded_file = st.sidebar.file_uploader("Choose an Excel, CSV, or JSON file", type=['xlsx', 'csv', 'json'])

# Textbox for brief description
dataset_brief = st.sidebar.text_area("💡 Enter a brief description of the dataset:")

if uploaded_file is not None:
    # Read dataset based on file format
    if uploaded_file.name.endswith('.xlsx'):
        df = pd.read_excel(uploaded_file)  # Ensure openpyxl is installed
    elif uploaded_file.name.endswith('.csv'):
        df = pd.read_csv(uploaded_file)
    elif uploaded_file.name.endswith('.json'):
        df = pd.read_json(uploaded_file)

    # Display first 20 columns of the dataset
    st.subheader("🔢 First 20 Columns of the Dataset")
    df_subset = df.iloc[:, :20]
    st.dataframe(df_subset)

    # Profile data types and categorize
    with st.expander("📊 Data Profile"):
        st.write("The data type for each column:")
        profile = data_profile(df_subset)
        profile_df = pd.DataFrame(list(profile.items()), columns=["Column", "Type"])
        st.dataframe(profile_df)

    # Missing values check
    with st.expander("❓ Missing Values"):
        missing_vals = missing_values(df_subset)
        st.dataframe(missing_vals)

    # Unique values check
    with st.expander("🔍 Unique Values"):
        unique_vals = unique_values(df_subset)
        st.dataframe(unique_vals)

    # Summary statistics
    with st.expander("📈 Summary Statistics"):
        summary = summary_stats(df_subset)
        st.dataframe(summary)

    # Cardinality check for categorical columns
    with st.expander("🔢 Cardinality of Categorical Columns"):
        cardinality_vals = cardinality(df_subset)
        st.dataframe(cardinality_vals)

    # Correlation matrix (Numerical columns only)
    with st.expander("📊 Correlation Matrix"):
        corr_matrix = correlation_matrix(df_subset)
        if isinstance(corr_matrix, str):
            st.write(corr_matrix)  # No numeric columns
        else:
            st.dataframe(corr_matrix)

    # Outlier detection (Numerical columns only)
    with st.expander("🔍 Outlier Detection"):
        outliers = detect_outliers(df_subset)
        if isinstance(outliers, str):
            st.write(outliers)  # No numeric columns
        else:
            st.dataframe(outliers)

    # Histogram for continuous variables
    with st.expander("📊 Histogram for Continuous Variables"):
        plot_histograms(df_subset)

    # Duplicate rows check
    with st.expander("♻️ Duplicate Rows Check"):
        duplicates = duplicate_rows(df_subset)
        st.write(f"Number of duplicate rows: {duplicates}")

    # Memory usage check
    with st.expander("💾 Memory Usage"):
        memory_usage_vals = memory_usage(df_subset)
        st.dataframe(memory_usage_vals)

    # Zero and constant values check
    with st.expander("🔢 Zero and Constant Values Check"):
        zero_vals, constant_vals = zero_constant_check(df_subset)
        st.write("Zero Value Counts per Column:")
        st.dataframe(zero_vals)
        st.write("Constant Value Columns (True means constant):")
        st.dataframe(constant_vals)

    # Time series analysis (if applicable)
    with st.expander("⏳ Time Series Analysis"):
        datetime_columns = df_subset.select_dtypes(include=['datetime64', 'datetime'])
        if not datetime_columns.empty:
            column_to_analyze = st.selectbox("Select date-time column to analyze:", datetime_columns.columns)
            if column_to_analyze:
                decomposition = time_series_analysis(df_subset, column_to_analyze)
                decomposition.plot()
                st.pyplot()
        else:
            st.write("No Date/Time columns detected for time series analysis.")

    # Button to send a smaller dataset and description to ChatGPT API
    if st.sidebar.button("Send Data"):
        # Check if description is provided
        if not dataset_brief:
            st.error("Please provide a brief description of the dataset.")
        else:
            # Call the OpenAI API function with limited rows and columns
            with st.spinner("Sending a smaller subset of data to ChatGPT..."):
                response = get_openai_response(df_subset, dataset_brief)
            
            # Display the response
            st.subheader("AI Powered Bot Response")
            st.write(response)

2024-10-08 03:08:59.743 
  command:

    streamlit run C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
